<a href="https://colab.research.google.com/github/hogo56/BertQA/blob/master/BERT_for_Humans_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## --Main System Config--
<Details><Summary>Global Config</Summary>
Put any global system configuration here

In [0]:
import os, sys
## Setup directory structure
! mkdir /content/lib
! mkdir -p /content/data
! mkdir -p /content/bert_output       # Maybe output to Google Drive for durability
sys.path.append('/content/lib')

! apt-get install vim > /dev/null

In [2]:
! pwd
! ls -l

/content
total 16
drwxr-xr-x 2 root root 4096 Dec 15 05:57 bert_output
drwxr-xr-x 2 root root 4096 Dec 15 05:57 data
drwxr-xr-x 2 root root 4096 Dec 15 05:57 lib
drwxr-xr-x 1 root root 4096 Dec 12 16:48 sample_data


### Runtime Parameters
EnableAllCode - There are code blocks here that should not be run with "Run All". By default EnableAllCode will set to False and those blocks will be excluded. If you want to run them individually for some reason set EnableAllCode True.<p>
DownloadBigFiles - There are GBs of files and downloads to make this run. If you are just wanting to spin up the Colab so you can SSH into it set DownloadBigFiles = False then Runtime -> RunAfter

In [0]:
EnableAllCode = True

In [0]:
EnableAllCode = False

In [0]:
DownloadBigFiles = True

In [0]:
DownloadBigFiles = False

## -- Setup --

### Machine Setup

--- Google Drive
<Details>There are several ways to provide access to your Google Drive from Colab. (What about the Drive FUSE wrapper?)<br>
I am not sure if this is the best. This mounts your Drive into the machine.<br>
I expect there will be a folder in the Drive that we all share.</Details>

In [7]:
## File link to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)   # true to reread drive
# Create a shorter shared directory name than one with a space
! ln -s '/content/gdrive/My Drive/bertqa' /content/bertqa

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
if EnableAllCode:
    ## Flush and unmount Google Drive
    # You probablyu won't do this but if you want to at some point click the play button
    drive.flush_and_unmount()

### SSH Setup
<Details> Shell access is not technically required but if you are like me and being able to look under the hood brings you comfort you can step throuth this.<br>
Thanks to Imad El Hanafi (https://imadelhanafi.com) for showing me how to do this.<p>
I think you will need to create a free account at https://ngrok.com/ for the SSH tunnel to work.</Details>

In [9]:
## Install sshd
! apt-get install -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:L2IuDmRtem6ndIbxXVQc9wI7t8Nnrrv57Ec3xnNeoPY root@e0f2060c6019 (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:YA0fsId2x+HpV71juKoqj20elKIOG4NpJFf7fDG2gU0 root@e0f2060c6019 (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:kkij3OU4iH2J/fYocli3gaqaPEGGCXQto98bTsplxQ4 root@e0f2060c6019 (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.


In [10]:
# 1 - setup ssh/user 

#Generate a random root password
import random, string
sshpass = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))

#Set root password
! echo root:$sshpass | chpasswd

print()
print("###### SSH Login Credentials ######")
print("username: root")
print("password: ", sshpass)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

###### SSH Login Credentials ######
username: root
password:  46U4SlVWMjv8RiV2Xn6l1jbakC4GYr


On Ngrok dashboard https://dashboard.ngrok.com/status you'll find the tcp address and the port you can connect to with your favorite ssh client. eg...

```
ssh root@0.tcp.ngrok.io -p [ngrok_port]
> then enter the username and password shown from the codeblock above

root@hostname:~ PS1=':\w\$ '     (or whatever you want)
:~# vim .bashrc        (if you want to change prompt PS1 there; also add cd/content to end)
:~# cd /content
:/content# ls -lh 
```
If you want to keep your ssh sessions from being disconnected you need to have your client send a KeepAlive about every 200 seconds.

In [0]:
# 2 - Download Ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
! rm ngrok-stable-linux-amd64.zip

In [12]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

Get your authtoken from https://dashboard.ngrok.com/auth
··········


In [0]:
if EnableAllCode:
    # When done, kill Ngrok
    !kill $(ps aux | grep './ngrok' | awk '{print $2}')

### Kaggle API
<Details>You will need a token to link the Colab instance to the API of your Kaggle account to get data, etc.<br>
Go to: https://www.kaggle.com/yourID/account and click on the "Create New API Token: button to get a file named kaggle.json and save it somewhere on your local drive.</Details>
<Details><Summary>More...</Summary>I am choosing to upload the kaggle.json file from local hard drive each time. Alternately, we could put it in a private location on your gdrive (not the shared folder.)<br>
To use Drive the location in the environ would be like /content/gdrive/kaggle/</Details>

In [14]:
## Link to Kaggle
from google.colab import files

# see if there is a kaggle.json file in gdrive
try:
    # see if auth file is in gdrive
    f = open("/content/gdrive/My Drive/kaggle/kaggle.json")
    ! chmod 600 "/content/gdrive/My Drive/kaggle/kaggle.json"      # file is likely already 600
    os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle/"
    ! ls -l "/content/gdrive/My Drive/kaggle/kaggle.json"
except IOError:
    # Have user upload file
    ! rm /content/kaggle.json  2> /dev/null
    print('Upload kaggle.json.')
    # The files.upload() command is failing sporatically with:
    #   TypeError: Cannot read property '_uploadFiles' of undefined (just run again)
    files.upload()
    ! chmod 600 /content/kaggle.json
    os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
    ! ls -l /content/kaggle.json

import kaggle

-rw------- 1 root root 66 Dec 15 02:55 '/content/gdrive/My Drive/kaggle/kaggle.json'


### Download Dataset and Support Files

In [0]:
## Competition Dataset  (5GB zipped)
if DownloadBigFiles:
    ! kaggle competitions list
    ! kaggle competitions download -c tensorflow2-question-answering -p /content/data

### Library Setup

In [21]:
## Stop Run All Execution here
# This is a really ugly way to get the Notebook to stop, any better ideas???

print()
print("### Stopping Notebook execution ###")
print("To continue Notebook put cursor on next cell and Runtime -> Run After")
print()
assert False


### Stopping Notebook execution ###
To continue Notebook put cursor on next cell and Runtime -> Run After



AssertionError: ignored

In [0]:
## Fix Nvidia            (make sure your Runtime type is python3 + GPU)

""" ToDo - Get Latest Nvidia Drivers """
if EnableAllCode:   # (this is just disabled because the code is not tested)
    # https://developer.nvidia.com/cudnn       (need to join developer program to get latest)

    # Extracts the cuDNN files from Drive folder directly to the VM CUDA folders
    !tar -xzvf bertqa/nvidia/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
    !chmod a+r /usr/local/cuda/include/cudnn.h

    # Now we check the version we already installed. Can comment this line on future runs
    !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2


In [0]:
## Copy lib files over from Google Drive
! cp -a /content/bertqa/lib/* lib/

In [0]:
if DownloadBigFiles:
    ## get BERT (this is unlikely to be the BERT-joint files needed for competition)
    # this version of BERT seems won't import as is. On line 88 of lib/bert/optimization.py
    #    change   tr.train.Optimizer to tf.keras.optimizers.Optimizer
    ! git clone https://github.com/google-research/bert.git
    ! mv bert lib

    # get some pretrained models  (I really  have no idea what these are or if useful)
    ! wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
    ! unzip cased_L-12_H-768_A-12.zip
    ! rm cased_L-12_H-768_A-12.zip

Are these the correct bert models we are supposed to be using?<br>
Maybe also look at https://github.com/tensorflow/models/tree/master/official/nlp/bert or,<br>
https://github.com/google-research/language/tree/master/language/question_answering/bert_joint

<Details><Summary>BERT Notes</Summary>
baseline_w_bert_translated_to_tf2_0 (next code block) comes from /dimitreoliveira with this warning:<br>
This baseline uses code that was migrated from TF1.x. Be aware that it contains use of tf.compat.v1, which is not permitted to be eligible for TF2.0 prizes in this competition. It is intended to be used as a starting point, but we're excited to see how much better you can do using TF2.0!<br>
https://www.kaggle.com/dimitreoliveira/using-tf-2-0-w-bert-on-nq-translated-to-tf2-0</Details>

<Details><Summary>Bert Updates</Summary>
On line 88 of lib/bert/optimization.py<br>
Change the parent class from tr.train.Optimizer to tf.keras.optimizers.Optimizer

In [0]:
## Load Libraries

%tensorflow_version 2.x 
import tensorflow
print(tensorflow.__version__)

import numpy as np
import pandas as pd
import tensorflow as tf

# import tf2_0_baseline_w_bert as tf2baseline # old script
import tf2_0_baseline_w_bert_translated_to_tf2_0 as tf2baseline # Oliviera's script

import bert.modeling as modeling
import bert.optimization as optimization
import bert.tokenization as tokenization

import json
import absl
from zipfile import ZipFile

## --Explore Environment--


In [0]:
if EnableAllCode:
    print('===== Python =====')
    print('Version: ', sys.version)
    ! pip -V
    print('===== CPU Info =====')
    !cat /proc/cpuinfo
    print('\n===== MEM Info =====')
    !cat /proc/meminfo
    print('\n===== printenv =====')
    ! printenv
    from tensorflow.python.client import device_lib
    print('\n===== List Local Devices =====')
    print(device_lib.list_local_devices())
    print()
    print("===== Dir of /:", *(os.listdir('/')), sep='\n')
    print("\n===== cwd: ", os.getcwd())                       # will be /content
    print('\n===== Files in', os.getcwd())
    for dirname, _, filenames in os.walk(os.getcwd()):
        for filename in filenames:
            print(os.path.join(dirname, filename))

In [0]:
if EnableAllCode:
    print('\n===== NVIDIA Info =====')
    !nvidia-smi
    print()
    !/usr/local/cuda/bin/nvcc --version

## --Misc Notes--

### File Transfer to Local Machine

In [0]:
# ## To upload files to Colab
# from google.colab import files
# uploaded = files.upload()
# print(uploaded)

In [0]:
# ## To download files from Colab
# from google.colab import files
# model.save('trained_model.h5')          # but probably better to save directly to Drive
# files.download('trained_model.h5')

### File Transfer from Web

In [0]:
# from urllib.request import urlretrieve

# def download(url, file):
#     if not os.path.isfile(file):
#         print("Downloading file... " + file + " ...")
#         urlretrieve(url,file)
#         print("File downloaded")

# download('Url of the file','Name of the file to be saved')
# print("All the files are downloaded")
# #If the downloaded file is a zip file than you can use below function to unzip it.
# def uncompress_features_labels(file):
#     if(os.path.isdir('data')):
#         print('Data extracted')
#     else:
#         with ZipFile(dir) as zipf:
#             zipf.extractall('data')

In [0]:
! printenv

### Prevent Disconnects
Colab periodically disconnects the browser.<br>
You have to save model checkpoints to Google Drive so you don't lose wor<br>
See: https://mc.ai/google-colab-drive-as-persistent-storage-for-long-training-runs/<br>
Something to try...<br>
Ctrl+Shift+i in browser and in console run this code...
```
function KeepAlive(){
    console.log("Maintaining Connection");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(KeepAlive,60000);
```
There have been reports of people having their GPU privileges suspended for letting processes run for over 12 hours. It seems that they may penalize you rather than just cutting you off.

## --Model Code--

Tensorflow flags are variables that can be passed around within the TF system. Every flag below has some context provided regarding what the flag is and how it's used.<p>
Most of these can be changed as desired, with the exception of the Special Flags at the bottom, which must stay as-is to work with the Kaggle back end.

In [0]:
DoNotGoPastHere()
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(absl.flags.FLAGS)

flags = absl.flags

flags.DEFINE_string(
    "bert_config_file", "/kaggle/input/bertjointbaseline/bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

flags.DEFINE_string("vocab_file", "/kaggle/input/bertjointbaseline/vocab-nq.txt",
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string(
    "output_dir", "outdir",
    "The output directory where the model checkpoints will be written.")

flags.DEFINE_string("train_precomputed_file", None,
                    "Precomputed tf records for training.")

flags.DEFINE_integer("train_num_precomputed", None,
                     "Number of precomputed tf records for training.")

flags.DEFINE_string(
    "output_prediction_file", "predictions.json",
    "Where to print predictions in NQ prediction format, to be passed to"
    "natural_questions.nq_eval.")

flags.DEFINE_string(
    "init_checkpoint", "/kaggle/input/bertjointbaseline/bert_joint.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_integer(
    "max_seq_length", 384,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_integer(
    "doc_stride", 128,
    "When splitting up a long document into chunks, how much stride to "
    "take between chunks.")

flags.DEFINE_integer(
    "max_query_length", 64,
    "The maximum number of tokens for the question. Questions longer than "
    "this will be truncated to this length.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")

flags.DEFINE_bool("do_predict", True, "Whether to run eval on the dev set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("predict_batch_size", 8,
                     "Total batch size for predictions.")

flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

flags.DEFINE_integer("save_checkpoints_steps", 1000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_integer(
    "n_best_size", 20,
    "The total number of n-best predictions to generate in the "
    "nbest_predictions.json output file.")

flags.DEFINE_integer(
    "verbosity", 1, "How verbose our error messages should be")

flags.DEFINE_integer(
    "max_answer_length", 30,
    "The maximum length of an answer that can be generated. This is needed "
    "because the start and end predictions are not conditioned on one another.")

flags.DEFINE_float(
    "include_unknowns", -1.0,
    "If positive, probability of including answers of type `UNKNOWN`.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_bool("use_one_hot_embeddings", False, "Whether to use use_one_hot_embeddings")

absl.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_bool(
    "verbose_logging", False,
    "If true, all of the warnings related to data processing will be printed. "
    "A number of warnings are expected for a normal NQ evaluation.")

flags.DEFINE_boolean(
    "skip_nested_contexts", True,
    "Completely ignore context that are not top level nodes in the page.")

flags.DEFINE_integer("task_id", 0,
                     "Train and dev shard to read from and write to.")

flags.DEFINE_integer("max_contexts", 48,
                     "Maximum number of contexts to output for an example.")

flags.DEFINE_integer(
    "max_position", 50,
    "Maximum context position for which to generate special tokens.")


## Special flags - do not change

flags.DEFINE_string(
    "predict_file", "/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl",
    "NQ json for predictions. E.g., dev-v1.1.jsonl.gz or test-v1.1.jsonl.gz")
flags.DEFINE_boolean("logtostderr", True, "Logs to stderr")
flags.DEFINE_boolean("undefok", True, "it's okay to be undefined")
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('HistoryManager.hist_file', '', 'kernel')

FLAGS = flags.FLAGS
FLAGS(sys.argv) # Parse the flags